In [1]:
import pandas as pd
import pyspark.pandas as ps
import requests
import pca
import json

from sqlalchemy import create_engine

from pyspark.sql.functions import pandas_udf, PandasUDFType, max, col, countDistinct, when, rank, lit
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
from pyspark.sql.window import Window

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from imblearn.under_sampling import ClusterCentroids


# from backend_spark.doris_common.doris_client import DorisClient
from functools import reduce
from pyspark.sql import DataFrame
from typing import Optional

In [41]:
df1_pandas = pd.read_csv("small2.csv")
df1_pandas

,TOOL_ID,TOOL_NAME,RUN_ID,EQP_NAME,CASE_INFO,PRODUCT_ID,PRODG1,OPER_NO,LOT_ID,WAFER_ID,...,LOWER_OUTLIER,UPPER_OUTLIER,RULES_ENABLED,ALARM_RULE,RESULT,STATUS,REGION,ERROR_MSG,STATISTIC_RESULT,VERSION
0,9289,PBT01_CGHA_4-34,351230,PBT01,2023-12-16,AFPZM801N.0A01,L2800Z3N,1V.PPB10,NBX265.000,NBX265-05,...,NaN,NaN,1,NaN,249.984800,NORMAL,LOWER_NORMAL,NaN,249.984800,3
1,9287,PBT01_CGHA_4-14,360329,PBT01,2023-12-16,AFPZM801N.0A01,L2800Z3N,1V.PPB10,NBX265.000,NBX265-06,...,NaN,NaN,1,NaN,249.984583,NORMAL,LOWER_NORMAL,NaN,249.984583,3
2,9279,PBT01_CLHA_4-12,360246,PBT01,2023-12-16,AFPZM801N.0A01,L2800Z3N,1V.PPB10,NBX265.000,NBX265-06,...,NaN,NaN,1,NaN,150.007500,NORMAL,LOWER_NORMAL,NaN,150.007500,4
3,9287,PBT01_CGHA_4-14,329563,PBT01,2023-10-13,AFPNR901N.0B0J,L2800Z2N,1V.PPB10,NBX221.100,NBX221-13,...,NaN,NaN,1,NaN,249.986250,NORMAL,LOWER_NORMAL,NaN,249.986250,3
4,9287,PBT01_CGHA_4-14,362271,PBT01,2023-12-20,AFPNR901N.0B0L,L2800Z2N,1V.PPB10,NBX293.200,NBX293-06,...,NaN,NaN,1,NaN,249.984583,NORMAL,LOWER_NORMAL,NaN,249.984583,3
5,9279,PBT01_CLHA_4-12,329480,PBT01,2023-10-13,AFPNR901N.0B0J,L2800Z2N,1V.PPB10,NBX221.100,NBX221-13,...,NaN,NaN,1,NaN,149.937083,NORMAL,LOWER_NORMAL,NaN,149.937083,4
6,9287,PBT01_CGHA_4-14,323876,PBT01,2023-09-28,AFPNR901N.0B0J,L2800Z2N,1V.PPB10,NBX220.150,NBX220-06,...,NaN,NaN,1,NaN,249.982500,NORMAL,LOWER_NORMAL,NaN,249.982500,3
7,9280,PBT01_CLHA_4-21,317196,PBT01,2023-09-25,AFPNR901N.0B0J,L2800Z2N,1V.PPB10,NBX219.130,NBX219-08,...,NaN,NaN,1,NaN,149.998261,NORMAL,LOWER_NORMAL,NaN,149.998261,4
8,9288,PBT01_CGHA_4-24,312691,PBT01,2023-09-13,AFPNR901N.0B0J,L2800Z2N,1V.PPB10,NBX220.040,NBX220-20,...,NaN,NaN,1,NaN,249.988750,NORMAL,UPPER_NORMAL,NaN,249.988750,3
9,9288,PBT01_CGHA_4-24,355016,PBT01,2023-12-16,AFPZM801N.0A01,L2800Z3N,1V.PPB10,NBX265.000,NBX265-08,...,NaN,NaN,1,NaN,249.982000,NORMAL,LOWER_NORMAL,NaN,249.982000,3


In [44]:
# df1_pandas['WAFER_ID'].unique()

In [43]:
df1 = ps.from_pandas(df1_pandas).to_spark()

/usr/local/spark/python/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/usr/local/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [45]:
############################################
######## 1. 客户只定义了bad_wafer = []是什么  ########
############################################
# 将传进来的BAD_WAFER, 用 | 连接起来，
# F.col('WAFER_ID').like('NDJ065%') | F.col('WAFER_ID').like('NDJ067%') 作为条件传入增加label
# 同时将isin模式也作为条件传入增加label

def get_label_single(df, bad_wafer):
    like_conditions = [f"col('WAFER_ID').like('{bad}')" for bad in bad_wafer]
    all_like_conditions = " | ".join(like_conditions)
    isin_conditions = "col('WAFER_ID').isin(bad_wafer)"
    df = df.withColumn('label', 
                when( eval(all_like_conditions) | eval(isin_conditions), int(1)).otherwise(int(0)))
    return df


############################################
## 2. 客户定义了bad_wafer = [] 和 good_wafer = []######
############################################
# 将传进来的BAD_WAFER, 用 | 连接起来，
# 将传进来的GOOD_WAFER, 也用 | 连接起来，
# 同时将isin模式也作为条件传入增加label

def get_label_double(df, bad_wafer, good_wafer):
    good_like_conditions = [f"col('WAFER_ID').like('{good}')" for good in good_wafer]
    all_good_like_conditions = " | ".join(good_like_conditions)
    good_isin_conditions = "col('WAFER_ID').isin(good_wafer)"

    bad_like_conditions = [f"col('WAFER_ID').like('{bad}')" for bad in bad_wafer]
    all_bad_like_conditions = " | ".join(bad_like_conditions)
    bad_isin_conditions = "col('WAFER_ID').isin(bad_wafer)"

    df = df.withColumn('label',  when(eval(all_good_like_conditions) | eval(good_isin_conditions), int(0)).when(eval(all_bad_like_conditions) | eval(bad_isin_conditions), int(1)).otherwise(222333))
    df = df.filter(df['label'] != int(222333))
    return df

In [46]:
good = ["NBX221-13","NBX220-06","NBX272-19","NBX219-08"]
bad  = ["NBX265-05","NBX265-06","NBX265-07","NBX265-08","NBX293-06","NBX293-07","NAZ998-21","NBX220-20"]

df1 = get_label_double(df1, bad, good)

In [47]:
df1.show()

+-------+---------------+------+--------+----------+--------------+--------+--------+----------+---------+--------------------+----------+---------------+----------+--------------------+-------------------+------------+-------------+--------------+-------+---------------+-------------------+-------------------+----------------+----------------+----------------+----------------+----------------+-------------+-------------+-------------+----------+-------------+------+------------+---------+----------------+-------+-----+
|TOOL_ID|      TOOL_NAME|RUN_ID|EQP_NAME| CASE_INFO|    PRODUCT_ID|  PRODG1| OPER_NO|    LOT_ID| WAFER_ID|         RECIPE_NAME|     INPUT|         WINDOW|STATISTICS|     parametric_name|         START_TIME|PARTITION_ID|STATISTIC_KEY|COLLECTION_KEY| SEQ_ID|PROCESS_TYPE_ID|         TIME_STAMP|    CALC_TIME_STAMP|          TARGET|   LOWER_WARNING|   UPPER_WARNING|  LOWER_CRITICAL|  UPPER_CRITICAL|LOWER_OUTLIER|UPPER_OUTLIER|RULES_ENABLED|ALARM_RULE|       RESULT|STATUS|   

In [48]:
bad_wafer_num = df1.filter("label == 1").select('WAFER_ID').distinct().count()
good_wafer_num = df1.filter("label == 0").select('WAFER_ID').distinct().count()
print(bad_wafer_num)
print(good_wafer_num)

8
4


In [50]:
############################################################################
##################################FDC数据预处理###############################
############################################################################
def _pre_process(df):
    """
    param df: 从数据库中读取出来的某个CASE数据
    return: 数据预处理，后面要根据实际情况统一添加
    """
    # 只选出会用到的列
    df = df.select('WAFER_ID', 'TOOL_ID', 'RUN_ID', 'EQP_NAME', 'PRODUCT_ID', 'PRODG1', 
                'TOOL_NAME', 'OPER_NO', 'parametric_name', 'STATISTIC_RESULT', 'label')
    # 剔除NA值
    df = df.filter(col('STATISTIC_RESULT').isNotNull())
    # 按照所有的行进行去重
    df1 = df.dropDuplicates()
    # 选最新的RUN
    df2 = df1.groupBy('WAFER_ID', 'OPER_NO', 'TOOL_ID').agg(max('RUN_ID').alias('RUN_ID'))
    df_run = df1.join(df2.dropDuplicates(subset=['WAFER_ID', 'OPER_NO', 'TOOL_ID', 'RUN_ID']),
                                on=['WAFER_ID', 'OPER_NO', 'TOOL_ID', 'RUN_ID'], how='inner')
    return df_run


def commonality_analysis(df_run, grpby_list):
    """
    param df_run: 数据预处理后的数据
    return: 共性分析后的结果， 返回bad wafer前五的组合
    """
    grps = (df_run.groupBy(grpby_list)
                        .agg(countDistinct('WAFER_ID').alias('wafer_count'),
              countDistinct('WAFER_ID', when(df_run['label'] == 0, 1)).alias('good_num'),
              countDistinct('WAFER_ID', when(df_run['label'] == 1, 1)).alias('bad_num'))
              .orderBy('bad_num', ascending=False))

    # 单站点+单腔室的情况
    if grps.count() == 1:
        return grps
    else: 
        grps = grps.filter(grps['bad_num'] > 0)
        window_sep = Window().orderBy(col("bad_num").desc())
        ranked_df = grps.withColumn("rank", rank().over(window_sep))
        grpss = ranked_df.filter(col("rank") <= 5).drop("rank")
    return grpss

In [51]:
df_run = _pre_process(df1)
print(df_run.count())

24


In [54]:
grpby_list = ['PRODG1', 'OPER_NO', 'TOOL_NAME']
common_res = commonality_analysis(df_run, grpby_list)
common_res.show()

+--------+--------+---------------+-----------+--------+-------+
|  PRODG1| OPER_NO|      TOOL_NAME|wafer_count|good_num|bad_num|
+--------+--------+---------------+-----------+--------+-------+
|L2800Z2N|1V.PPB10|PBT01_CGHA_4-14|          4|       3|      1|
|L2800Z2N|1V.PPB10|PBT01_CLHA_4-12|          4|       3|      1|
|L2800Z3N|1V.PPB10|PBT01_CLHA_4-12|          1|       0|      1|
|L2800Z1N|1V.PPB10|PBT01_CGHA_4-14|          1|       0|      1|
|L2800Z3N|1V.PPB10|PBT01_CGHA_4-13|          1|       0|      1|
|L2800Z3N|1V.PPB10|PBT01_CLHA_4-11|          1|       0|      1|
|L2800Z1N|1V.PPB10|PBT01_CLHA_4-12|          1|       0|      1|
|L2800Z3N|1V.PPB10|PBT01_CLHA_4-21|          1|       0|      1|
|L2800Z3N|1V.PPB10|PBT01_CLHA_4-31|          1|       0|      1|
|L2800Z2N|1V.PPB10|PBT01_CLHA_4-11|          1|       0|      1|
|L2800Z2N|1V.PPB10|PBT01_CGHA_4-13|          1|       0|      1|
|L2800Z2N|1V.PPB10|PBT01_CLHA_4-21|          2|       1|      1|
|L2800Z3N|1V.PPB10|PBT01_